In [1]:
import tensorflow as tf
print(f"tf.__version__: {tf.__version__}")
if tf.test.gpu_device_name(): 
    print(f"Default GPU Device:{tf.test.gpu_device_name()}")
else:
    print("Please install GPU version of TF")
from os import getcwd
import os.path
from os import listdir
import numpy as np
import pandas as pd
from pathlib import Path
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

tf.__version__: 2.1.3
Default GPU Device:/device:GPU:0


## Directories

In [2]:
! ls

dataset_generation_for_models.ipynb	       photometry_lib.ipynb
lstm_experiment_1.ipynb			       simple_autoencoder.ipynb
lstm_experiment_2.ipynb			       training_conv
models					       training_lstm
notebook_dataset_generation_for_models_output  training_lstm2


In [3]:
datapath = Path("/home/baroncelli/phd/rtapipe/analysis/notebook_dataset_generation_for_models_output")
print(datapath)
assert datapath.is_dir() == True

/home/baroncelli/phd/rtapipe/analysis/notebook_dataset_generation_for_models_output


In [4]:
currentdir = getcwd()
currentdir

'/home/baroncelli/phd/rtapipe/analysis'

In [5]:
outdir = Path(currentdir).joinpath("notebook_lstm_output")
outdir

PosixPath('/home/baroncelli/phd/rtapipe/analysis/notebook_lstm_output')

In [23]:
class WindowGenerator:
    
    def __init__(self, input_width, label_width, shift):

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

        self.train_df = None
        self.val_df = None
        self.test_df = None

        self.data = {}


    def loadClassDataFromDir(dataDir, sampleClass, sampleSize, windowSize, deleteRows=0):
        
        dataframes = [  pd.read_csv(str(dataDir.joinpath(f)), header=0) for f in listdir(dataDir) if dataDir.joinpath(f).is_file()]
        
        dataframe = pd.concat(dataframes, axis=0)

        if deleteRows > 0:
            dataframe = dataframe[:-deleteRows]

        numberOfSamples = int(dataframe.shape[0]/sampleSize)
        bkgTimespan = round( (dataframe.shape[0]*windowSize)/60/60, 2)

        print(f"Found {len(dataframes)} files! Each file has: \n\t{dataframes[0].shape[0]} rows (samples) \n\t{dataframes[0].shape[1]} columns!")

        print(f"Temporal size = {bkgTimespan} h. Each sample is equivalent to {bkgTimespan/numberOfSamples*60*60} seconds")


        ## USE A WINDOW GENERATOR HERE?

        dataframe.head()
        dataframe = pd.DataFrame(dataframe["COUNTS"].values.reshape(numberOfSamples, sampleSize))
        dataframe[sampleSize] = sampleClass
        print(dataframe.shape)
        print(dataframe.head(2))



        return dataframe


    def addData(self, label, label_columns=None):


        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}

        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}        
        
        
        
    def split(self):

        train_data, other_data, train_labels, other_labels = train_test_split(
            data, labels, test_size=0.4, random_state=69
        )

        test_data, val_data, test_labels, val_labels = train_test_split(
            other_data, other_labels, test_size=0.1, random_state=69
        )        

        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df


        
    def __repr__(self):
        return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])
    def split_window(self, features):

        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                          axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def plot(self, model=None, plot_col='COUNTS', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
          label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
    label_col_index = plot_col_index

    if label_col_index is None:
    continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
              edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
    predictions = model(inputs)
    plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                marker='X', edgecolors='k', label='Predictions',
                c='#ff7f0e', s=64)

    if n == 0:
    plt.legend()

    plt.xlabel('Time [h]')

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

        ds = ds.map(self.split_window)

        return ds

    @property
    def train(self):
    return self.make_dataset(self.train_df)

    @property
    def val(self):
    return self.make_dataset(self.val_df)

    @property
    def test(self):
    return self.make_dataset(self.test_df)

    @property
    def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.train))
        # And cache it for next time
        self._example = result
    return result

In [34]:
SAMPLE_SIZE = 180
WINDOW_SIZE = 10

In [35]:
bkgDataDir = datapath.joinpath("run0406_ID000126_OK_bkg_only","csv")
bkgSamples = processDataDir(bkgDataDir, 0, SAMPLE_SIZE, WINDOW_SIZE)

Found 100 files! Each file has: 
	180 rows (samples) 
	5 columns!
Temporal size = 50.0 h. Each sample is equivalent to 1800.0 seconds
(100, 181)
   0    1    2    3    4    5    6    7    8    9    ...  171  172  173  174  \
0  293  235  279  224  277  256  272  252  245  260  ...  253  252  279  252   
1  251  267  273  288  282  212  283  301  233  255  ...  262  244  267  268   

   175  176  177  178  179  180  
0  249  244  274  248  239    0  
1  251  251  269  270  250    0  

[2 rows x 181 columns]


In [36]:
bkgSrcDataDir = datapath.joinpath("run0406_ID000126_OK_bkg_src","csv")
bkgSrcSamples = processDataDir(bkgSrcDataDir, 1, SAMPLE_SIZE, WINDOW_SIZE)

Found 100 files! Each file has: 
	180 rows (samples) 
	5 columns!
Temporal size = 50.0 h. Each sample is equivalent to 1800.0 seconds
(100, 181)
   0    1    2    3    4    5    6    7    8    9    ...  171  172  173  174  \
0  243  251  286  257  274  249  284  296  275  267  ...  286  244  256  285   
1  273  237  280  262  255  267  236  258  286  268  ...  258  282  250  265   

   175  176  177  178  179  180  
0  300  285  274  269  262    1  
1  246  272  274  284  275    1  

[2 rows x 181 columns]


In [ ]:
w1 = WindowGenerator(input_width=10, label_width=1, shift=10, label_columns=['COUNTS'], val_df=val_df, train_df=train_df, test_df=test_df)
w1

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Input, Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import Model

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [9]:
modelLSTM = Sequential()
modelLSTM.add(LSTM(32, input_shape=(x_train.shape[1], x_train.shape[2])))
modelLSTM.add(Dropout(rate=0.3))
modelLSTM.add(RepeatVector(x_train.shape[1]))
modelLSTM.add(LSTM(32, return_sequences=True))
modelLSTM.add(Dropout(rate=0.3))
modelLSTM.add(TimeDistributed(Dense(x_train.shape[2])))

modelLSTM.compile(optimizer='adam', loss='mae')
modelLSTM.summary()

NameError: name 'x_train' is not defined

In [ ]:
checkpoint_path_lstm = Path("./training_lstm/cp.ckpt")
checkpoint_path_lstm.mkdir(exist_ok=True, parents=True)

In [ ]:
cp_callback_lstm  = keras.callbacks.ModelCheckpoint(filepath=str(checkpoint_path_lstm), save_weights_only=True, verbose=1)


In [ ]:
epochs=20

In [ ]:
modelLSTMHistory = modelLSTM.fit(x_train, x_train, epochs=1, batch_size=128, validation_split=0.1, verbose=1, callbacks=[cp_callback_lstm])